In [1]:
import nltk

#nltk.download()

# Librerías para PNLIngred_Description
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import es_dep_news_trf # https://spacy.io/usage/models
import language_tool_python
from spacy.matcher import Matcher
from spacy.util import filter_spans


# Otras librerías
import os, shutil
import pandas as pd
from collections import Counter
import json
import re

from pattern.es import pluralize
from pattern.es import singularize

#!python -m spacy download es_dep_news_trf
#conda install -c conda-forge spacy-model-es_dep_news_trf
#!pip install xlrd
#pip install language-tool-python

In [2]:
#!python -m spacy download es_core_news_sm / es_core_news_lg

In [7]:
textfile = os.getcwd() + "\\Converted results\\" + "Rajas_crema.txt"

try:
    with open(textfile) as file:
        fileContent = file.read()
        file.close()
except FileNotFoundError:
     print(f'The file {textfile} does not exist')
else:
    parsed_text = parse_text(fileContent)

parsed_text
    

[['hola', 'hola', 'PROPN', 'proper noun', 'dep', ],
 ['súper', 'súper', 'PROPN', 'proper noun', 'flat', ],
 ['bienvenidos',
  'bienvenido',
  'ADJ',
  'adjective',
  'flat',
  Gender=Masc|Number=Plur],
 ['a', 'a', 'ADP', 'adposition', 'case', ],
 ['ipsilan', 'ipsilan', 'PROPN', 'proper noun', 'nmod', ],
 ['el',
  'el',
  'DET',
  'determiner',
  'det',
  Definite=Def|Gender=Masc|Number=Sing|PronType=Art],
 ['día', 'día', 'NOUN', 'noun', 'obl', Gender=Masc|Number=Sing],
 ['de', 'de', 'ADP', 'adposition', 'case', ],
 ['hoy', 'hoy', 'ADV', 'adverb', 'advmod', ],
 ['voy',
  'ir',
  'AUX',
  'auxiliary',
  'aux',
  Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin],
 ['a', 'a', 'ADP', 'adposition', 'mark', ],
 ['preparar', 'preparar', 'VERB', 'verb', 'ROOT', VerbForm=Inf],
 ['unas',
  'uno',
  'DET',
  'determiner',
  'det',
  Definite=Ind|Gender=Fem|Number=Plur|PronType=Art],
 ['fáciles', 'fácil', 'ADJ', 'adjective', 'amod', Number=Plur],
 ['y', 'y', 'CCONJ', 'coordinating conjunction'

In [8]:
tokenized_words = get_tokenized_words(parsed_text)
tokenized_words

[[0, 'súper', 'súper', 'PROPN', 'proper noun', 'flat', ],
 [1,
  'bienvenidos',
  'bienvenido',
  'ADJ',
  'adjective',
  'flat',
  Gender=Masc|Number=Plur],
 [2, 'a', 'a', 'ADP', 'adposition', 'case', ],
 [3, 'ipsilan', 'ipsilan', 'PROPN', 'proper noun', 'nmod', ],
 [4,
  'el',
  'el',
  'DET',
  'determiner',
  'det',
  Definite=Def|Gender=Masc|Number=Sing|PronType=Art],
 [5, 'día', 'día', 'NOUN', 'noun', 'obl', Gender=Masc|Number=Sing],
 [6, 'de', 'de', 'ADP', 'adposition', 'case', ],
 [7, 'hoy', 'hoy', 'ADV', 'adverb', 'advmod', ],
 [8,
  'voy',
  'ir',
  'AUX',
  'auxiliary',
  'aux',
  Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin],
 [9, 'a', 'a', 'ADP', 'adposition', 'mark', ],
 [10, 'preparar', 'preparar', 'VERB', 'verb', 'ROOT', VerbForm=Inf],
 [11,
  'unas',
  'uno',
  'DET',
  'determiner',
  'det',
  Definite=Ind|Gender=Fem|Number=Plur|PronType=Art],
 [12, 'fáciles', 'fácil', 'ADJ', 'adjective', 'amod', Number=Plur],
 [13, 'y', 'y', 'CCONJ', 'coordinating conjunctio

In [9]:
tool = language_tool_python.LanguageToolPublicAPI('es-MX')

In [10]:
verbs_stop_lemma = ["estar", "tener", "basar"]
nouns_stop_text = ["personas", "vuelta", "manita"]
morph_attributes = {"INTERSECTS": ["VerbForm=Inf"]}

pattern = [
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # agregarle las rajas
           {'POS': 'DET', 'OP': '{0}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}}
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # agregarle las rajas
           {'POS': 'DET', 'OP': '{0,}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'ADP', 'OP': '{0,}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
     #      {'POS': 'PRON', 'OP': '?'},
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # dejar aquí 15 minutos
           {'POS': 'ADV', 'OP': '?'},
           {'POS': 'NUM', 'OP': '?'},
           {'POS': 'NOUN', 'OP': '{1}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}}
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # desvenar estos chilitos guajillos
           {'POS': 'DET', 'OP': '?'},
           {'POS': 'NOUN', 'OP': '{1}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'ADJ', 'OP': '?'},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # poner poquito agua,
           {'POS': 'ADJ', 'OP': '{1}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # poner poquito de aceite,
           {'POS': 'ADV', 'OP': '{1}'},
           {'POS': 'ADP', 'OP': '{1}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # agregarle las rajas
           {'POS': 'DET', 'OP': '{0,}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'DET', 'OP': '?'},
           {'POS': 'NOUN', 'OP': '?', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'CCONJ', 'OP': '{1}'},
           {'POS': 'DET', 'OP': '{0,}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # dejar que hierva bien,
           {'POS': 'SCONJ', 'OP': '{1}'},
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}}, 
           {'POS': 'ADV', 'OP': '{1}'},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # meter en una bolsa de plastico
           {'POS': 'ADP', 'OP': '{1}'},
           {'POS': 'DET', 'OP': '{1}'}, 
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'ADP', 'OP': '{1}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # tapar con un secador
           {'POS': 'ADP', 'OP': '{1}'},
           {'POS': 'DET', 'OP': '{1}'}, 
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
    [
           {'POS': 'VERB', 'OP': '{1}', "LEMMA": {"NOT_IN": verbs_stop_lemma}, "MORPH": morph_attributes}, # poner una taza y media de crema
           {'POS': 'DET', 'OP': '{1}'},    
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
           {'POS': 'CCONJ', 'OP': '{1}'},
           {'POS': 'NUM', 'OP': '{1}'},
           {'POS': 'ADP', 'OP': '{1}'},
           {'POS': 'NOUN', 'OP': '{1,}', "DEP": {"NOT_IN": ["compound"]}, 'TEXT': {"NOT_IN": nouns_stop_text}},
    ],
     
]

In [11]:
nlp = spacy.load("es_dep_news_trf")

# instantiate a Matcher instance
matcher = Matcher(nlp.vocab)
matcher.add("Verb phrase", pattern)

doc = nlp(fileContent)

# call the matcher to find matches 
matches = matcher(doc)
spans = [doc[start:end] for _, start, end in matches]

value_list = filter_spans(spans)
value_list

[chuparse los dedos,
 cocer dos jitomates,
 usar queso seco,
 meter la tortilla,
 rellenar tacos dorados,
 agregar aguacate,
 regálame un like,
 chuparse los dedos,
 hacer pollo]

In [12]:
tool = language_tool_python.LanguageToolPublicAPI('es-MX')

procedure_to_string = ""

for number, sentence in enumerate(value_list):
    my_mistakes = []
    my_corrections = []

    text = str(sentence)
    
    matches = tool.check(text)
    
    for rules in matches:      
        if rules.ruleId == 'MORFOLOGIK_RULE_ES': # no nos importa que reemplace ingredientes
            continue
        else:
            my_mistakes.append(text[rules.offset : rules.errorLength + rules.offset])
            my_corrections.append(rules.replacements[0])
            
               
    if len(my_corrections) == 0:
        procedure_to_string += "\n" + str(number + 1) + ".- " + text + "."
        continue
    
    new_text = text
    
  #  for word in text.split():
    for i, mistake in enumerate(my_mistakes):
        new_text = new_text.replace(my_mistakes[i], my_corrections[i])

    procedure_to_string += "\n" + str(number + 1) + ".- " + new_text + "."

In [13]:
# Crea el archivo de ingredientes
with open('recipe procedure.txt', 'w') as f:
    f.write("PROCEDIMIENTO:\n")
    f.write(procedure_to_string)

    f.close()